<a href="https://colab.research.google.com/github/PriyankaGona/machinelearning-assignments/blob/master/hw1_problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import mnist
import numpy as np

In [0]:
# convert digits to binary values
def to_categorical(y,nb_classes):
  output = []
  for i in y:
    temp = np.zeros(nb_classes)
    temp[i] = 1
    output.append(temp)
  output = np.array(output)
  output = output.reshape(y.shape[0],nb_classes)
  return output

In [0]:
# intitalizing weights and bias to zero
def init(dim):
    W = np.zeros(shape=(dim, num_classes))
    b = 0
    return W, b

In [0]:
# sigmoid activation function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [0]:
# Binary cross entropy loss function
def binary_crossentropy(A, Y, batch_size):
    m = batch_size
    loss = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))
    loss = np.squeeze(loss)
    return loss

In [0]:
# backward propagation
def backward_propagate(A, X, Y, batch_size):
    m = batch_size
    # gradient computation
    dW = (1 / m) * np.dot(X, (2 * (A-Y) * A * (1-A)).T)
    db = (1 / m) * np.sum(2 * (A-Y) * A * (1-A))

    return dW, db

In [0]:
# forward porpagation
def forward_propogate(W, b, X):
    linear_transformation = np.dot(W.T, X) + b
    output = sigmoid(linear_transformation)
    return output

In [0]:
# mini-batch stochastic gradient descent
def sgd(W, b, X, Y, epochs, learning_rate):

    for j in range(epochs):
        training_loss = []
        for i in range(0, X.shape[1], batch_size):
            x_train_mini = X.T[i:i+batch_size]
            y_train_mini = Y.T[i:i+batch_size]
            output = forward_propogate(W,b,x_train_mini.T)
            loss = binary_crossentropy(output,y_train_mini.T,batch_size)
            dW, db = backward_propagate(output,x_train_mini.T,y_train_mini.T,batch_size)

            # gradient descent
            W = W - learning_rate * dW
            b = b - learning_rate * db

            training_loss.append(loss)
        epochLoss=sum(training_loss)/len(training_loss)
        print("Epoch {}/{}\t - loss : {}".format(j+1,epochs,round(epochLoss,4)))
    return W, b

In [0]:
# calculating accuracy of test data after training
def predict(W, b, X):

    m = X.shape[1]
    y_pred = np.zeros((1, m))
    W = W.reshape(X.shape[0], num_classes)
    A = forward_propogate(W,b,X)

    for i in range(A.shape[1]):
        y_pred[0, i] = 1 if A[0, i] > 0.5 else 0

    return y_pred

In [0]:
def model(X_train, Y_train, X_test, Y_test, epochs=50, learning_rate=0.1):

    y_pred_train = Y_train
    y_pred_test = Y_test

    # shape of predict_y_train and predict_y_test is 60000,10 similar to that of y_train and y_test categorical values
    y_pred_train = to_categorical(y_pred_train, 10)
    y_pred_test = to_categorical(y_pred_test, 10)

    for i in range(0, 10):
        # set the digit to classify
        classifier_digit = i
        print("\nClassifier: {}".format(classifier_digit))
        print("=======================================")
        # modify training labels to create single class classification
        y_train_mod = np.array(Y_train)
        y_train_mod = np.where(y_train_mod == classifier_digit, 1, 0)

        y_test_mod = np.array(Y_test)
        y_test_mod = np.where(y_test_mod == classifier_digit, 1, 0)

        # core of the model
        W, b = init(X_train.shape[0])
        W, b = sgd(W, b, X_train, y_train_mod, epochs, learning_rate)

        # predicting the values based on trained weights and bias values
        Y_prediction_train = predict(W, b, X_train)
        Y_prediction_test = predict(W, b, X_test)

        # we will store ones in the corresponding to indices to create array to similar to categorical data of y_train
        y_pred_train[:, [i]] = Y_prediction_train.T
        y_pred_test[:, [i]] = Y_prediction_test.T

        # train and test classifier accuracy for each digit
        print("\nAccuracy of classifer in training " + str(i) + " : {} %".format(round((100 - np.mean(np.abs(Y_prediction_train - y_train_mod)) * 100),4)))
        print("Accuracy of classifer in testing" + str(i) + " : {} %".format(round((100 - np.mean(np.abs(Y_prediction_test - y_test_mod)) * 100),4)))
        print("\n")

    # coverting y_train from 60000,1 to 60000,10 i.e. to categorical
    Y_train = to_categorical(Y_train, 10)
    Y_test = to_categorical(Y_test, 10)

    # overall test and train classifier accuracy of the network
    print("Overall accuracy in Training: {} %".format(round((100 - np.mean(np.abs(y_pred_train - Y_train)) * 100),4)))
    print("Overall accuracy in Testing: {} %".format(round((100 - np.mean(np.abs(y_pred_test - Y_test)) * 100),4)))

In [15]:
# variables
batch_size = 32
num_classes = 1
epochs = 12

# Image Dimensions of MNIST
rows = 28
cols = 28

# loading MNIST dataset using keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# reshaping dataset
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]).T
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]).T

# Normalization
x_train = x_train / 255
x_test = x_test / 255

y_train = y_train.T
y_test = y_test.T


model(x_train, y_train, x_test, y_test, epochs=12,learning_rate=0.01)


Classifier: 0
Epoch 1/12	 - loss : 0.1685
Epoch 2/12	 - loss : 0.0963
Epoch 3/12	 - loss : 0.0807
Epoch 4/12	 - loss : 0.0723
Epoch 5/12	 - loss : 0.0669
Epoch 6/12	 - loss : 0.063
Epoch 7/12	 - loss : 0.0599
Epoch 8/12	 - loss : 0.0575
Epoch 9/12	 - loss : 0.0555
Epoch 10/12	 - loss : 0.0538
Epoch 11/12	 - loss : 0.0524
Epoch 12/12	 - loss : 0.0511

Accuracy of classifer in training 0 : 98.9017 %
Accuracy of classifer in testing0 : 99.14 %



Classifier: 1
Epoch 1/12	 - loss : 0.1528
Epoch 2/12	 - loss : 0.0852
Epoch 3/12	 - loss : 0.0708
Epoch 4/12	 - loss : 0.0635
Epoch 5/12	 - loss : 0.0589
Epoch 6/12	 - loss : 0.0557
Epoch 7/12	 - loss : 0.0532
Epoch 8/12	 - loss : 0.0513
Epoch 9/12	 - loss : 0.0498
Epoch 10/12	 - loss : 0.0485
Epoch 11/12	 - loss : 0.0474
Epoch 12/12	 - loss : 0.0465

Accuracy of classifer in training 1 : 98.9 %
Accuracy of classifer in testing1 : 99.19 %



Classifier: 2
Epoch 1/12	 - loss : 0.2319
Epoch 2/12	 - loss : 0.1514
Epoch 3/12	 - loss : 0.1311
Epoch 4